## Dependencies

In [ ]:
# Confirmed needed dependencies
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import random
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim
import time

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

# Dependency for Heat Mapper
import gmaps


# Add config.py file with the following variables and cooresponding Zillow API keys
from config import Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta
from config import google_API_Key


################# ONGOING EDITS TO REQUIREMENTS.MD #################
###### IF ANY ERRORS OCCUR RELATING TO MODULES OR CONFIG.PY #######
### REFER TO requirements.md TO ENSURE YOU ARE PROPERLY SETUP ####

## File inputs/outputs

In [ ]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 20         #### Set radius in miles

points = 40000        #### Set number of lat,lon points to generate

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [ ]:
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "mymap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Visuals/" + mapOutputFile)

## Convert Coordinates to Residential Addresses

In [ ]:
#########################
##### Yuta's Blocks #####
#########################

##### Geopy Nominatim API #####
geopy.geocoders.options.default_user_agent = "ut-group-EPIC"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

In [ ]:
# Test API - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== Test Home Response:")
response = requests.get(url, params=params_1).json()
pp.pprint(response)
print("\n" + "="*60 + "\n")

In [ ]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

In [ ]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [ ]:
##### Loop Request API for Addresses / Append to lists #####
# Make sure to import time

query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
lat = []
lon = []
neighborhood = []

counter = 1
numRequests = latlon_df["lat"].count()
rSuccess = []
rFailure = []

print(f"Processing {numRequests} Requests...")

# Nominatim API Request

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }

    time.sleep(1.1)
    response = requests.get(query_url, params=params).json()

    if response['type'] == 'house' or response['type'] == 'yes':
        lat.append(response['lat'])
        lon.append(response['lon'])
        
        try:
            postcode.append(response['address']['postcode'])
        except (KeyError, IndexError):
            postcode.append("NA")
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        
        print(f"Processed Record {counter} of {numRequests}.")
        rSuccess.append(counter)
        counter += 1
        
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        rFailure.append(counter)
        counter += 1
        
print(f"Finished Requests !!!")

In [ ]:
print("Request Results:")
print("Success #:" + str(len(rSuccess)))
print("Skipped #:" + str(len(rFailure)))

In [ ]:
# Create dataframe with addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
})

# Clean up Dataframe Columns before output (Drop incomplete zipcodes, Highway streets, and Null house # or streets)
address_df = address_df[address_df['zipcode'].str.len() == 5]
address_df = address_df[address_df['zipcode'].apply(lambda x: len(str(x)) > 3)]
address_df = address_df[address_df['street'].str.contains("Highway") == False]
address_df = address_df[address_df['house #'].str.contains("NA") == False]
address_df = address_df[address_df['street'].str.contains("NA") == False]
address_df.dtypes

In [ ]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [ ]:
# Map out CSV with gmplot

addressList_csv = "./Data/addressList.csv"

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(addressList_csv["lat"], addressList_csv["lon"], 'red', size=20, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [ ]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

In [ ]:
### TEST BLCOK FOR FUNCTION ELEMENTS OF csvDFappend ###

colNames = masterDFIMPORTclean.columns
newCol = "zipcode"
for value in colNames:
    print(value)
    if value == newCol:
        print("Cannot append column that matches an existing column name")
        break
        
if len(address_df["zipcode"]) != len(masterDFIMPORTclean):
    print("Cannot append column that is not the same length as existing dataframe")  

In [ ]:
# Tiny sample to work with looping without exhausting API call limits
addressListTiny_csv = "./Data/addressListTiny.csv"

address_sample = pd.read_csv(addressListTiny_csv)
address_df = pd.DataFrame(address_sample)
print(len(address_df))

In [ ]:
# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

In [ ]:
addressDF = csvDF(addressList_csv)
print(len(addressDF))

In [ ]:
############### LOOPING FUNCTION FULLY OPERATIONAL ################
####### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY #######

# Zillow API call function
def zCall(API, index, address, zipcode):
    APIkey = API[index]
    zillow_data = ZillowWrapper(APIkey)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    return result

# List containers for collected property data
zid = []
addresses = []
alats = []
alons = []
valuation = []
sqft = []

# List of Zillow API keys to loop through due to daily API call limits
zAPIs = [Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, 
         Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, 
         Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta]
index = 0
    
for row, home in addressDF.iterrows():
    address = str(addressDF["house #"][row]) + " " + str(addressDF["street"][row])
    addresses.append(address)
    zipcode = addressDF["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")
    
    result = None
    try:
        try:
            result = zCall(zAPIs, index, address, zipcode)
            print(f"{row} Success!")
        except KeyError:  ### ERROR FOR API CALL LIMIT EXCEEDED ###
            print(f"KeyError has occurred for {address}, {zipcode} (index {row}).")
            index += 1
            print(f"Proceeding to API[{index}]")
            if index >= len(zAPIs):
                print(f"API[{index}] does not exist. Need more API keys to complete analysis.")
                break
            result = zCall(zAPIs, index, address, zipcode)

    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        sqft.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)

    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)

    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)

    try:
        zsqft = int(result.home_size)
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)

In [ ]:
print(len(addresses))

## Calculate Value per Sqft

In [ ]:
valsqft = []
for row, value in enumerate(valuation):
    try:
        vsqft = round((valuation[row] / sqft[row]), 2)
        valsqft.append(vsqft)
    except:
        print("Cannot perform math with NoneType")
        valsqft.append(None)

## Master Dataframe Creation

In [ ]:
# Checking to ensure lists are appropriate lengths
print(len(zid))
print(len(alats))
print(len(alons))
print(len(addresses))
print(len(valuation))
print(len(sqft))
print(len(valsqft))

# Referring back to addressList_csv generated dataframe for relevant info
addressDF.head()

In [ ]:
# FULL SAMPLE FROM 40,000 RANDOM LAT, LON GENERATION

masterDF = pd.DataFrame({
    "Zillow ID": zid,
    "address": addresses,
    "zipcode": addressDF["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "sqft": sqft,
    "value sqft": valsqft,
    "neighborhood": addressDF["neighborhood"],
})
print(len(masterDF))
masterDF.head(10)

In [ ]:
masterDFclean = masterDF.dropna(how="any", subset=["Zillow ID"])
len(masterDFclean)

In [ ]:
# masterDF to csv
masterDFclean.to_csv(masterData_csv, index=False, header=True)
masterDFclean.head(30)

In [ ]:
### CONTINUE HERE READ IN NEW CSV TO CLEAN

In [ ]:
masterDFdrops = masterDFclean.dropna(how="any", subset=["value sqft"])
len(masterDFdrops)

In [ ]:
masterDFdrops = masterDFdrops.dropna(how="any", subset=["sqft"])


In [ ]:
masterDFdrops.sort_values(by="value sqft", ascending=False)
# masterDFIMPORTclean.to_csv("./Data/masterDFIMPORTclean.csv", index=False, header=True)
# masterDFIMPORTclean.head(10)

## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig